# Summary Evaluation Demo (Standalone Version)

This notebook demonstrates how to evaluate financial/business summaries using ROUGE, BERTScore, and stylometric analysis.

**This is a standalone version with all code embedded - no external src/ dependencies required.**

## Local RoBERTa Model Setup (Optional)

For faster BERTScore evaluation and offline usage, download RoBERTa-large model locally:

```bash
# Run once to download model files (~1.4GB)
python setup_roberta.py

# Verify it works
python test_local_roberta.py
```

The notebook will automatically detect and use local model files if `roberta-large/` directory exists.

## Setup

In [ ]:
# Core imports
import json
import re
import string
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Iterator, Dict, Any, List, Optional
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from rouge_score import rouge_scorer
from scipy.spatial.distance import jensenshannon
import nltk

print("✓ Imports successful")

## Text Utilities

Basic text processing functions for tokenization and stylometric analysis:

In [ ]:
def ensure_nltk_data():
    """Download required NLTK data if not present."""
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=True)
    
    try:
        nltk.data.find('tokenizers/punkt_tab')
    except LookupError:
        nltk.download('punkt_tab', quiet=True)


def tokenize_sentences(text: str) -> List[str]:
    """Tokenize text into sentences."""
    ensure_nltk_data()
    return nltk.sent_tokenize(text)


def tokenize_words(text: str) -> List[str]:
    """Tokenize text into words."""
    ensure_nltk_data()
    return nltk.word_tokenize(text)


def count_tokens(text: str) -> int:
    """Count number of tokens in text."""
    return len(tokenize_words(text))


def get_function_words() -> set:
    """Get set of common function words for stylometric analysis."""
    return {
        'the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'have', 'i',
        'it', 'for', 'not', 'on', 'with', 'he', 'as', 'you', 'do', 'at',
        'this', 'but', 'his', 'by', 'from', 'they', 'we', 'say', 'her', 'she',
        'or', 'an', 'will', 'my', 'one', 'all', 'would', 'there', 'their',
        'what', 'so', 'up', 'out', 'if', 'about', 'who', 'get', 'which', 'go',
        'me', 'when', 'make', 'can', 'like', 'time', 'no', 'just', 'him', 'know',
        'take', 'people', 'into', 'year', 'your', 'good', 'some', 'could', 'them',
        'see', 'other', 'than', 'then', 'now', 'look', 'only', 'come', 'its', 'over'
    }


def get_pronouns() -> set:
    """Get set of pronouns for stylometric analysis."""
    return {
        'i', 'you', 'he', 'she', 'it', 'we', 'they',
        'me', 'him', 'her', 'us', 'them',
        'my', 'your', 'his', 'her', 'its', 'our', 'their',
        'mine', 'yours', 'hers', 'ours', 'theirs',
        'myself', 'yourself', 'himself', 'herself', 'itself', 'ourselves', 'themselves'
    }


def _count_syllables(word: str) -> int:
    """Count syllables in a word (approximation)."""
    word = word.lower()
    vowels = 'aeiouy'
    syllable_count = 0
    previous_was_vowel = False
    
    for char in word:
        is_vowel = char in vowels
        if is_vowel and not previous_was_vowel:
            syllable_count += 1
        previous_was_vowel = is_vowel
    
    # Adjust for silent 'e'
    if word.endswith('e'):
        syllable_count -= 1
    
    # Ensure at least 1 syllable
    return max(1, syllable_count)


def calculate_flesch_kincaid_grade(text: str) -> float:
    """Calculate Flesch-Kincaid grade level."""
    sentences = tokenize_sentences(text)
    words = tokenize_words(text)
    
    if not sentences or not words:
        return 0.0
    
    # Count syllables (approximation)
    syllable_count = sum(_count_syllables(word) for word in words)
    
    num_sentences = len(sentences)
    num_words = len(words)
    
    if num_sentences == 0 or num_words == 0:
        return 0.0
    
    # FK grade formula
    grade = 0.39 * (num_words / num_sentences) + 11.8 * (syllable_count / num_words) - 15.59
    
    return max(0.0, grade)


def get_punctuation_counts(text: str) -> dict:
    """Count different types of punctuation."""
    return {
        'comma': text.count(','),
        'period': text.count('.'),
        'semicolon': text.count(';'),
        'colon': text.count(':'),
        'exclamation': text.count('!'),
        'question': text.count('?'),
        'total': sum(1 for c in text if c in string.punctuation)
    }

print("✓ Text utilities loaded")

## I/O Utilities

Functions for loading and processing JSON data files:

In [ ]:
def load_json_files(data_dir: str = "data") -> Iterator[Dict[str, Any]]:
    """Load all JSON files from a directory."""
    data_path = Path(data_dir)
    
    if not data_path.exists():
        raise FileNotFoundError(f"Data directory not found: {data_dir}")
    
    json_files = sorted(data_path.glob("*.json"))
    
    if not json_files:
        raise FileNotFoundError(f"No JSON files found in {data_dir}")
    
    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                # Add filename for tracking
                data['_source_file'] = json_file.name
                yield data
            except json.JSONDecodeError as e:
                print(f"Warning: Failed to parse {json_file.name}: {e}")
                continue


def extract_fields(record: Dict[str, Any]) -> Dict[str, Any]:
    """Extract all relevant fields from a record."""
    # Extract metadata
    metadata = record.get('metadata', {})
    
    return {
        # Core content
        'document_content': record.get('document_content', ''),
        'expected_summary': record.get('expected_summary', ''),
        'generated_summary': record.get('generated_summary', ''),
        
        # Metadata
        'document_title': record.get('document_title', ''),
        'link': record.get('link', ''),
        'author': metadata.get('author', ''),
        'sector': metadata.get('sector', ''),
        'region': metadata.get('region', ''),
        'date': metadata.get('date', ''),
        'wire_id': metadata.get('wire_id', ''),
        'subject_codes': metadata.get('subject_codes', []),
        
        # Model info
        'prompt_type': record.get('prompt_type', ''),
        'model_used': record.get('model_used', ''),
        'export_timestamp': record.get('export_timestamp', ''),
        
        # Tracking
        '_source_file': record.get('_source_file', ''),
    }


def infer_persona(record: Dict[str, Any]) -> str:
    """Infer persona from record metadata."""
    prompt_type = record.get('prompt_type', '').lower()
    author = record.get('author', '').lower()
    sector = record.get('sector', '').lower()
    
    # Formal analyst style: research notes, detailed analysis
    if any(keyword in prompt_type for keyword in ['research', 'analyst', 'note']):
        return 'formal_analyst'
    
    if any(keyword in author for keyword in ['analyst', 'economist', 'strategist', 'phd', 'cfa']):
        return 'formal_analyst'
    
    # Journalist style: morning summaries, news-style
    if any(keyword in prompt_type for keyword in ['morning', 'summary', 'brief', 'update']):
        return 'journalist'
    
    # Enthusiast style: quick takes, highlights
    if any(keyword in prompt_type for keyword in ['quick', 'highlight', 'takeaway']):
        return 'enthusiast'
    
    # Default to journalist for financial/business content
    return 'journalist'


def load_all_records(data_dir: str = "data") -> List[Dict[str, Any]]:
    """Load and process all JSON files from directory."""
    records = []
    
    for raw_record in load_json_files(data_dir):
        # Extract all fields
        extracted = extract_fields(raw_record)
        
        # Infer persona
        extracted['persona'] = infer_persona(extracted)
        
        records.append(extracted)
    
    return records

print("✓ I/O utilities loaded")

## Content Metrics Calculator

Calculate ROUGE, BERTScore, and BLEURT metrics:

In [ ]:
class ContentMetricsCalculator:
    """Calculate content quality metrics for summaries."""
    
    def __init__(self, config: Dict[str, Any]):
        """
        Initialize metrics calculator.
        
        Args:
            config: Configuration dictionary with content metrics settings
        """
        self.config = config
        self.content_config = config.get('content', {})
        
        # Initialize ROUGE
        if self.content_config.get('use_rouge', True):
            rouge_types = ['rouge1', 'rouge2', 'rougeLsum']
            self.rouge_scorer = rouge_scorer.RougeScorer(rouge_types, use_stemmer=True)
        else:
            self.rouge_scorer = None
        
        # BERTScore model
        self.bertscore_model = self.content_config.get('bertscore_model', 'roberta-large')
        
        # BLEURT checkpoint
        self.bleurt_checkpoint = self.content_config.get('bleurt_checkpoint', 'BLEURT-20-D12')
        self.bleurt_scorer = None  # Lazy load
    
    def _init_bleurt(self):
        """Lazy initialization of BLEURT scorer."""
        if self.bleurt_scorer is None and self.content_config.get('use_bleurt', True):
            try:
                from bleurt import score
                self.bleurt_scorer = score.BleurtScorer(self.bleurt_checkpoint)
            except Exception as e:
                print(f"Warning: BLEURT initialization failed: {e}")
                self.bleurt_scorer = False
    
    def calculate_rouge(self, reference: str, hypothesis: str) -> Dict[str, float]:
        """Calculate ROUGE scores."""
        if not self.rouge_scorer:
            return {}
        
        scores = self.rouge_scorer.score(reference, hypothesis)
        
        return {
            'rouge1_f': scores['rouge1'].fmeasure,
            'rouge2_f': scores['rouge2'].fmeasure,
            'rougeLsum_f': scores['rougeLsum'].fmeasure,
            'rouge1_r': scores['rouge1'].recall,
            'rouge2_r': scores['rouge2'].recall,
            'rougeLsum_r': scores['rougeLsum'].recall,
        }
    
    def calculate_bertscore(self, reference: str, hypothesis: str) -> Optional[float]:
        """Calculate BERTScore."""
        if not self.content_config.get('use_bertscore', True):
            return None
        
        try:
            import bert_score
            P, R, F1 = bert_score.score(
                [hypothesis],
                [reference],
                model_type=self.bertscore_model,
                lang='en',
                rescale_with_baseline=True,
                verbose=False
            )
            return F1.item()
        except Exception as e:
            print(f"Warning: BERTScore calculation failed: {e}")
            return None
    
    def calculate_bleurt(self, reference: str, hypothesis: str) -> Optional[float]:
        """Calculate BLEURT score."""
        if not self.content_config.get('use_bleurt', True):
            return None
        
        self._init_bleurt()
        
        if not self.bleurt_scorer:
            return None
        
        try:
            scores = self.bleurt_scorer.score(
                references=[reference],
                candidates=[hypothesis]
            )
            return scores[0]
        except Exception as e:
            print(f"Warning: BLEURT calculation failed: {e}")
            return None
    
    def calculate_all_metrics(
        self,
        source_text: str,
        reference: str,
        hypothesis: str
    ) -> Dict[str, Any]:
        """Calculate all content metrics for a single example."""
        metrics = {}
        
        # ROUGE
        rouge_scores = self.calculate_rouge(reference, hypothesis)
        metrics.update(rouge_scores)
        
        # BERTScore
        bertscore_f1 = self.calculate_bertscore(reference, hypothesis)
        metrics['bertscore_f1'] = bertscore_f1
        
        # BLEURT
        bleurt_score = self.calculate_bleurt(reference, hypothesis)
        metrics['bleurt'] = bleurt_score
        
        # Token counts and compression ratio
        src_tokens = count_tokens(source_text)
        hyp_tokens = count_tokens(hypothesis)
        gold_tokens = count_tokens(reference)
        
        metrics['src_tokens'] = src_tokens
        metrics['hyp_tokens'] = hyp_tokens
        metrics['gold_tokens'] = gold_tokens
        metrics['compression_ratio'] = hyp_tokens / src_tokens if src_tokens > 0 else 0.0
        
        # Composite score
        rougeLsum_f = metrics.get('rougeLsum_f', 0.0)
        bertscore = bertscore_f1 if bertscore_f1 is not None else 0.0
        bleurt = bleurt_score if bleurt_score is not None else 0.0
        
        # Normalize BLEURT to 0-1 range (BLEURT typically ranges from -1 to 1)
        bleurt_normalized = (bleurt + 1) / 2 if bleurt is not None else 0.0
        
        metrics['content_quality'] = (
            0.4 * rougeLsum_f +
            0.3 * bertscore +
            0.3 * bleurt_normalized
        )
        
        return metrics

print("✓ ContentMetricsCalculator loaded")

## Style Analyzer

Extract stylometric features and calculate persona similarity:

In [ ]:
class StyleAnalyzer:
    """Analyze stylometric features and calculate persona similarity."""
    
    def __init__(self, config: Dict[str, Any]):
        """
        Initialize style analyzer.
        
        Args:
            config: Configuration dictionary with style settings and persona paths
        """
        self.config = config
        self.style_config = config.get('style', {})
        self.personas = config.get('personas', {})
        self.centroids = {}
        self.function_words = get_function_words()
        self.pronouns = get_pronouns()
    
    def extract_stylometric_features(self, text: str) -> np.ndarray:
        """Extract stylometric feature vector from text."""
        if not text.strip():
            return np.zeros(10)
        
        # Tokenize
        sentences = tokenize_sentences(text)
        words = tokenize_words(text)
        words_lower = [w.lower() for w in words]
        
        if not words:
            return np.zeros(10)
        
        # Feature 1: Function word rate
        function_word_count = sum(1 for w in words_lower if w in self.function_words)
        function_word_rate = function_word_count / len(words)
        
        # Feature 2: Average sentence length
        avg_sentence_length = len(words) / len(sentences) if sentences else 0
        
        # Feature 3: Type-token ratio (vocabulary diversity)
        type_token_ratio = len(set(words_lower)) / len(words) if words else 0
        
        # Feature 4-7: Punctuation rates
        punct_counts = get_punctuation_counts(text)
        total_chars = len(text)
        comma_rate = punct_counts['comma'] / total_chars if total_chars > 0 else 0
        period_rate = punct_counts['period'] / total_chars if total_chars > 0 else 0
        exclamation_rate = punct_counts['exclamation'] / total_chars if total_chars > 0 else 0
        question_rate = punct_counts['question'] / total_chars if total_chars > 0 else 0
        
        # Feature 8: Pronoun rate
        pronoun_count = sum(1 for w in words_lower if w in self.pronouns)
        pronoun_rate = pronoun_count / len(words)
        
        # Feature 9: Flesch-Kincaid grade
        fk_grade = calculate_flesch_kincaid_grade(text)
        fk_grade_normalized = min(fk_grade / 20.0, 1.0)  # Normalize to 0-1
        
        # Feature 10: Average word length
        avg_word_length = sum(len(w) for w in words) / len(words)
        avg_word_length_normalized = min(avg_word_length / 10.0, 1.0)
        
        features = np.array([
            function_word_rate,
            avg_sentence_length / 50.0,  # Normalize (assume max 50 words/sentence)
            type_token_ratio,
            comma_rate * 100,  # Scale up
            period_rate * 100,
            exclamation_rate * 100,
            question_rate * 100,
            pronoun_rate,
            fk_grade_normalized,
            avg_word_length_normalized
        ])
        
        return features
    
    def build_persona_centroids(self, force_rebuild: bool = False) -> Dict[str, np.ndarray]:
        """Build stylometric centroids for each persona from their corpus files."""
        cache_path = Path('outputs/persona_centroids.json')
        
        # Try to load from cache
        if not force_rebuild and cache_path.exists():
            with open(cache_path, 'r') as f:
                cached = json.load(f)
                self.centroids = {k: np.array(v) for k, v in cached.items()}
                return self.centroids
        
        centroids = {}
        
        for persona_id, corpus_path in self.personas.items():
            if not Path(corpus_path).exists():
                print(f"Warning: Persona corpus not found: {corpus_path}")
                continue
            
            # Load persona corpus
            with open(corpus_path, 'r', encoding='utf-8') as f:
                corpus_text = f.read()
            
            # Split into samples (assume samples separated by double newlines)
            samples = [s.strip() for s in corpus_text.split('\n\n') if s.strip()]
            
            if not samples:
                print(f"Warning: No samples found for persona {persona_id}")
                continue
            
            # Extract features from each sample
            feature_vectors = []
            for sample in samples:
                features = self.extract_stylometric_features(sample)
                feature_vectors.append(features)
            
            # Calculate centroid (mean of all samples)
            centroid = np.mean(feature_vectors, axis=0)
            centroids[persona_id] = centroid
        
        # Cache centroids
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        with open(cache_path, 'w') as f:
            json.dump({k: v.tolist() for k, v in centroids.items()}, f, indent=2)
        
        self.centroids = centroids
        return centroids
    
    def calculate_style_similarity(
        self,
        text: str,
        persona_id: Optional[str]
    ) -> Optional[float]:
        """Calculate stylometric similarity between text and persona centroid."""
        if not self.style_config.get('use_stylometric_similarity', True):
            return None
        
        if persona_id is None:
            return None
        
        # Ensure centroids are built
        if not self.centroids:
            self.build_persona_centroids()
        
        if persona_id not in self.centroids:
            print(f"Warning: No centroid for persona {persona_id}")
            return None
        
        # Extract features from text
        text_features = self.extract_stylometric_features(text)
        persona_centroid = self.centroids[persona_id]
        
        # Normalize features to probability distributions (add small epsilon to avoid zeros)
        epsilon = 1e-10
        text_dist = text_features + epsilon
        text_dist = text_dist / text_dist.sum()
        
        centroid_dist = persona_centroid + epsilon
        centroid_dist = centroid_dist / centroid_dist.sum()
        
        # Calculate Jensen-Shannon divergence
        js_divergence = jensenshannon(text_dist, centroid_dist)
        
        # Convert to similarity (1 - divergence)
        # JS divergence is in [0, 1] for probability distributions
        similarity = 1.0 - js_divergence
        
        return float(similarity)
    
    def calculate_style_metrics(
        self,
        hypothesis: str,
        persona_id: Optional[str]
    ) -> Dict[str, Any]:
        """Calculate all style metrics for a summary."""
        style_similarity = self.calculate_style_similarity(hypothesis, persona_id)
        style_skipped = 1 if (persona_id is None or style_similarity is None) else 0
        
        return {
            'style_similarity': style_similarity,
            'style_skipped': style_skipped,
            'style_fidelity': style_similarity  # Alias for composite metric
        }

print("✓ StyleAnalyzer loaded")

## Load Data

Load JSON files from data/ directory:

In [ ]:
# Load all JSON files
records = load_all_records('data')

print(f"✓ Loaded {len(records)} records")
print(f"\nPersonas: {set(r['persona'] for r in records)}")
print(f"Sectors: {set(r['sector'] for r in records)}")

## Examine One Example

In [ ]:
# Look at first record
example = records[0]

print("📰 Title:", example['document_title'])
print("📝 Sector:", example['sector'])
print("👤 Author:", example['author'])
print("🎭 Persona (auto-detected):", example['persona'])
print("🤖 Model:", example['model_used'])
print("📋 Prompt:", example['prompt_type'])
print("\n" + "="*80)
print("\n📄 SOURCE (first 300 chars):")
print(example['document_content'][:300] + "...")
print("\n" + "="*80)
print("\n🎯 EXPECTED SUMMARY:")
print(example['expected_summary'][:500] + "..." if len(example['expected_summary']) > 500 else example['expected_summary'])
print("\n" + "="*80)
print("\n🤖 GENERATED SUMMARY:")
print(example['generated_summary'][:500] + "..." if len(example['generated_summary']) > 500 else example['generated_summary'])

## ROUGE Metrics

ROUGE measures n-gram overlap:
- **ROUGE-1**: Word overlap
- **ROUGE-2**: 2-word phrase overlap  
- **ROUGE-L**: Longest common subsequence

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

reference = example['expected_summary']
generated = example['generated_summary']

scores = scorer.score(reference, generated)

print("📊 ROUGE Scores for Example 1:\n")
for metric_name, metric_scores in scores.items():
    print(f"{metric_name.upper()}:")
    print(f"  Precision: {metric_scores.precision:.4f}")
    print(f"  Recall:    {metric_scores.recall:.4f}")
    print(f"  F1:        {metric_scores.fmeasure:.4f}\n")

## BERTScore

BERTScore uses neural embeddings to measure semantic similarity:

## Local Model Setup

Check if RoBERTa-large model is available locally and configure environment:

In [ ]:
import os

# Check if local RoBERTa model exists
local_model_path = Path("roberta-large")

if local_model_path.exists() and (local_model_path / "config.json").exists():
    # Set HF_HOME to use local model files
    os.environ['HF_HOME'] = str(local_model_path.parent.absolute())
    print(f"✓ Using local RoBERTa-large model from: {local_model_path.absolute()}")
    print("  This will speed up BERTScore evaluation and works offline.")
else:
    print("ℹ Local RoBERTa model not found")
    print("  BERTScore will download from HuggingFace on first use.")
    print("  To use local model: run 'python setup_roberta.py'")
print()

In [ ]:
try:
    import bert_score
    
    print("Calculating BERTScore (may take a minute on first run)...\n")
    
    P, R, F1 = bert_score.score(
        [example['generated_summary']], 
        [example['expected_summary']],
        model_type='roberta-large',
        lang='en',
        rescale_with_baseline=True,
        verbose=False
    )
    
    print(f"📊 BERTScore:")
    print(f"  Precision: {P.item():.4f}")
    print(f"  Recall:    {R.item():.4f}")
    print(f"  F1:        {F1.item():.4f}")
    
    print(f"\n💡 BERTScore F1 of {F1.item():.4f} indicates ", end="")
    if F1.item() > 0.9:
        print("excellent semantic similarity")
    elif F1.item() > 0.85:
        print("good semantic similarity")
    else:
        print("fair semantic similarity")
    
    bertscore_available = True
    
except Exception as e:
    print(f"⚠ BERTScore demo skipped: {e}")
    print("Install with: pip install bert-score")
    bertscore_available = False

## Stylometric Analysis

Extract writing style features:

In [ ]:
persona_config = {
    'personas': {
        'formal_analyst': 'data/personas/formal_analyst.txt',
        'journalist': 'data/personas/journalist.txt',
        'enthusiast': 'data/personas/enthusiast.txt'
    }
}
style_analyzer = StyleAnalyzer(persona_config)

# Extract features
features = style_analyzer.extract_stylometric_features(example['generated_summary'])

feature_names = [
    'Function word rate',
    'Avg sentence length (norm)',
    'Type-token ratio',
    'Comma rate',
    'Period rate',
    'Exclamation rate',
    'Question rate',
    'Pronoun rate',
    'FK grade (norm)',
    'Avg word length (norm)'
]

print("📐 Stylometric Features:\n")
for name, value in zip(feature_names, features):
    print(f"{name:30s}: {value:.4f}")

## Persona Similarity

In [ ]:
# Build persona centroids
print("Building persona centroids...\n")
centroids = style_analyzer.build_persona_centroids(force_rebuild=True)
print(f"✓ Built centroids for {len(centroids)} personas\n")

# Calculate similarity to each persona
print(f"Summary persona: {example['persona']}")
print(f"\n📊 Style Similarity to Each Persona:\n")

similarities = {}
for persona_id in centroids.keys():
    similarity = style_analyzer.calculate_style_similarity(example['generated_summary'], persona_id)
    similarities[persona_id] = similarity
    marker = "✓" if persona_id == example['persona'] else " "
    print(f"  {marker} {persona_id:20s}: {similarity:.4f}")

best_match = max(similarities, key=similarities.get)
print(f"\n🏆 Best match: {best_match}")
if best_match == example['persona']:
    print("✓ Correct!")
else:
    print(f"✗ Mismatch (expected {example['persona']})")

## Complete Evaluation

Run full evaluation on all records:

In [ ]:
results = []

print(f"Evaluating {len(records)} records...\n")

content_config = {
    'use_rouge': True,
    'use_bertscore': bertscore_available,
    'bertscore_model': 'roberta-large',
    'use_bleurt': False
}

content_calc = ContentMetricsCalculator({'content': content_config})

for i, rec in enumerate(records, 1):
    print(f"  [{i}/{len(records)}] {rec['document_title'][:50]}...")
    
    # Calculate content metrics
    content_metrics = content_calc.calculate_all_metrics(
        source_text=rec['document_content'],
        reference=rec['expected_summary'],
        hypothesis=rec['generated_summary']
    )
    
    # Calculate style similarity
    style_sim = style_analyzer.calculate_style_similarity(
        rec['generated_summary'], 
        rec['persona']
    )
    
    # Calculate overall quality (70% content + 30% style)
    content_quality = content_metrics.get('content_quality', 0.0)
    style_score = style_sim if style_sim is not None else 0.0
    overall_quality = (0.7 * content_quality) + (0.3 * style_score)
    
    results.append({
        'file': rec['_source_file'],
        'title': rec['document_title'][:40],
        'sector': rec['sector'],
        'persona': rec['persona'],
        'model': rec['model_used'],
        **content_metrics,
        'style_similarity': style_sim,
        'overall_quality': overall_quality
    })

df = pd.DataFrame(results)
print("\n✓ Evaluation complete!")

## Results Summary

In [ ]:
print("\n📊 EVALUATION RESULTS\n")
print("="*80)

# Display key columns
display_cols = ['title', 'persona', 'overall_quality', 'rouge1_f', 'rougeLsum_f']
if bertscore_available and 'bertscore_f1' in df.columns:
    display_cols.append('bertscore_f1')
display_cols.append('style_similarity')

print(df[display_cols].to_string(index=False))

print("\n" + "="*80)
print("\n📈 SUMMARY STATISTICS\n")

print("Overall Quality (70% content + 30% style):")
print(f"  Overall Quality:  {df['overall_quality'].mean():.4f} ± {df['overall_quality'].std():.4f}")
print(f"  Min: {df['overall_quality'].min():.4f}, Max: {df['overall_quality'].max():.4f}")

print(f"\nContent Quality:")
print(f"  Content Quality:  {df['content_quality'].mean():.4f} ± {df['content_quality'].std():.4f}")
print(f"  ROUGE-1 F1:       {df['rouge1_f'].mean():.4f} ± {df['rouge1_f'].std():.4f}")
print(f"  ROUGE-Lsum F1:    {df['rougeLsum_f'].mean():.4f} ± {df['rougeLsum_f'].std():.4f}")

if bertscore_available and 'bertscore_f1' in df.columns:
    print(f"  BERTScore F1:     {df['bertscore_f1'].mean():.4f} ± {df['bertscore_f1'].std():.4f}")

print(f"\nStyle Fidelity:")
print(f"  Style Similarity: {df['style_similarity'].mean():.4f} ± {df['style_similarity'].std():.4f}")

print(f"\nBy Persona:")
for persona in sorted(df['persona'].unique()):
    persona_df = df[df['persona'] == persona]
    print(f"  {persona:20s}: {len(persona_df)} items")
    print(f"    Overall:  {persona_df['overall_quality'].mean():.3f}")
    print(f"    Content:  {persona_df['content_quality'].mean():.3f}")
    print(f"    ROUGE-L:  {persona_df['rougeLsum_f'].mean():.3f}")
    print(f"    Style:    {persona_df['style_similarity'].mean():.3f}")

print(f"\nBy Sector:")
for sector in sorted(df['sector'].unique()):
    sector_df = df[df['sector'] == sector]
    print(f"  {sector:20s}: {len(sector_df)} items, overall={sector_df['overall_quality'].mean():.3f}")

## Visualization

Plot the results:

In [ ]:
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    sns.set_style('whitegrid')
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Overall Quality vs ROUGE
    for persona in df['persona'].unique():
        persona_df = df[df['persona'] == persona]
        axes[0, 0].scatter(persona_df['rougeLsum_f'], persona_df['overall_quality'], 
                       label=persona, alpha=0.7, s=150)
    axes[0, 0].set_xlabel('ROUGE-Lsum F1')
    axes[0, 0].set_ylabel('Overall Quality')
    axes[0, 0].set_title('ROUGE vs Overall Quality')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Content vs Style
    for persona in df['persona'].unique():
        persona_df = df[df['persona'] == persona]
        axes[0, 1].scatter(persona_df['content_quality'], persona_df['style_similarity'], 
                       label=persona, alpha=0.7, s=150)
    axes[0, 1].set_xlabel('Content Quality')
    axes[0, 1].set_ylabel('Style Similarity')
    axes[0, 1].set_title('Content Quality vs Style Fidelity')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Overall quality by persona
    df.boxplot(column='overall_quality', by='persona', ax=axes[1, 0])
    axes[1, 0].set_title('Overall Quality by Persona')
    axes[1, 0].set_ylabel('Overall Quality')
    axes[1, 0].set_xlabel('Persona')
    
    # Overall quality by sector
    df.boxplot(column='overall_quality', by='sector', ax=axes[1, 1])
    axes[1, 1].set_title('Overall Quality by Sector')
    axes[1, 1].set_ylabel('Overall Quality')
    axes[1, 1].set_xlabel('Sector')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('outputs/evaluation_viz.png', dpi=150, bbox_inches='tight')
    print("\n✓ Saved visualization to outputs/evaluation_viz.png")
    plt.show()
    
except ImportError:
    print("⚠ Matplotlib not available")
    print("Install with: pip install matplotlib seaborn")

## Next Steps

1. **Add more JSON files** to `data/` directory
2. **Run full evaluation**: `python -m src.eval_runner`
3. **Check results**: `outputs/per_item_metrics.csv`
4. **Customize personas**: Edit `data/personas/*.txt`

### Interpreting Scores

**Overall Quality (Combined Metric):**
- Overall Quality > 0.6: Good summary (70% content + 30% style)
- Overall Quality > 0.7: Excellent summary

**Content Quality:**
- Content Quality > 0.6: Good content coverage (weighted ROUGE + BERTScore)
- ROUGE-Lsum F1 > 0.6: Good overlap with reference
- BERTScore F1 > 0.85: Strong semantic match

**Style Fidelity:**
- Style similarity > 0.7: Good persona match
- Style similarity > 0.8: Strong persona match

### Metric Weighting

The overall quality metric combines:
- **70% Content Quality**: Weighted average of ROUGE (40%), BERTScore (30%), BLEURT (30%)
- **30% Style Similarity**: Jensen-Shannon divergence of stylometric features